In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# 1. Chargement des données
# Assurez-vous que le fichier 'diamonds.csv' est dans le même répertoire ou spécifiez le chemin complet.
try:
    df = pd.read_csv('diamonds.csv', index_col=0)
except FileNotFoundError:
    print("Erreur: Le fichier 'diamonds.csv' n'a pas été trouvé. Veuillez vérifier le chemin d'accès.")
    exit()

# 2. Nettoyage des données
# Suppression des lignes où les dimensions (x, y, z) sont nulles, car elles sont invalides
df = df.drop(df[ (df["x"]==0) | (df["y"]==0) | (df["z"]==0) ].index)
print(f"Taille du jeu de données après nettoyage: {len(df)} échantillons.")

# 3. Définition des features (X) et de la cible (y)
# X inclut toutes les autres colonnes pour prédire 'price'
X = df.drop('price', axis=1)
y = df['price']

# 4. Définition des colonnes pour le prétraitement
categorical_features = ['cut', 'color', 'clarity']
numerical_features = ['carat', 'depth', 'table', 'x', 'y', 'z']

# 5. Création des transformateurs
# Les variables catégorielles sont encodées (One-Hot Encoding)
# Les variables numériques sont mises à l'échelle (StandardScaler)
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('num', StandardScaler(), numerical_features)
    ],
    remainder='drop'
)

# 6. Création du pipeline
# Le pipeline applique le prétraitement puis la régression linéaire
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# 7. Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Taille de l'ensemble d'entraînement: {len(X_train)} échantillons.")
print(f"Taille de l'ensemble de test: {len(X_test)} échantillons.")

# 8. Entraînement du modèle
print("\nEntraînement du modèle de Régression Linéaire...")
model_pipeline.fit(X_train, y_train)
print("Entraînement terminé.")

# 9. Prédiction sur l'ensemble de test
y_pred = model_pipeline.predict(X_test)

# 10. Évaluation du modèle
# Les métriques R2 et MSE sont essentielles pour la comparaison
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\n--- Résultats de la Régression Linéaire Classique (Baseline) ---")
print(f"Erreur Quadratique Moyenne (MSE) : {mse:.2f}")
print(f"Racine de l'Erreur Quadratique Moyenne (RMSE) : {rmse:.2f}")
print(f"Coefficient de Détermination (R²) : {r2:.4f}")

# Pour la comparaison: Récupérer les coefficients pour interprétation
final_model = model_pipeline.named_steps['regressor']

# Attention: la récupération des noms de features après OneHotEncoder est complexe.
# Pour simplifier, nous affichons le nombre de coefficients
num_coefficients = len(final_model.coef_)
print(f"\nLe modèle a appris {num_coefficients} coefficients (un par feature encodée).")
print("Rappel: le R² représente la proportion de la variance de la cible ('price') expliquée par les features.")

Taille du jeu de données après nettoyage: 53920 échantillons.
Taille de l'ensemble d'entraînement: 43136 échantillons.
Taille de l'ensemble de test: 10784 échantillons.

Entraînement du modèle de Régression Linéaire...
Entraînement terminé.

--- Résultats de la Régression Linéaire Classique (Baseline) ---
Erreur Quadratique Moyenne (MSE) : 1290368.13
Racine de l'Erreur Quadratique Moyenne (RMSE) : 1135.94
Coefficient de Détermination (R²) : 0.9195

Le modèle a appris 26 coefficients (un par feature encodée).
Rappel: le R² représente la proportion de la variance de la cible ('price') expliquée par les features.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# --- 1. CHARGEMENT ET NETTOYAGE DES DONNÉES ---
try:
    df = pd.read_csv('diamonds.csv', index_col=0)
except FileNotFoundError:
    print("Erreur: Le fichier 'diamonds.csv' n'a pas été trouvé.")
    exit()

# Suppression des lignes invalides (x, y ou z == 0)
df = df.drop(df[ (df["x"]==0) | (df["y"]==0) | (df["z"]==0) ].index)

# --- 2. DÉFINITION DES FEATURES (X) ET DES CIBLES (Y) ---
# X: Features (variables d'entrée)
# Dans le dataset original, 'carat' est une feature mais était prédit dans votre DL.
# Pour simuler la structure de votre DL, nous allons utiliser toutes les colonnes SAUF les deux cibles.
# Features: 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z'
X = df.drop(['price', 'carat'], axis=1)

# Y: Cibles (variables de sortie/à prédire)
# Cibles: 'price', 'carat'
Y = df[['price', 'carat']]

# --- 3. PRÉTRAITEMENT (Pipeline) ---
categorical_features = ['cut', 'color', 'clarity']
numerical_features = ['depth', 'table', 'x', 'y', 'z']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('num', StandardScaler(), numerical_features)
    ],
    remainder='passthrough' # Ne pas inclure 'price' ou 'carat'
)

# --- 4. MODÈLE ML CLASSIQUE MULTI-CIBLE ---
# Nous utilisons MultiOutputRegressor pour gérer les deux cibles (price et carat) simultanément
base_regressor = LinearRegression()
multi_target_regressor = MultiOutputRegressor(base_regressor)

# Création du pipeline final
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', multi_target_regressor)
])

# --- 5. ENTRAÎNEMENT ET TEST ---
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model_pipeline.fit(X_train, Y_train)
Y_pred_train = model_pipeline.predict(X_train)
Y_pred_test = model_pipeline.predict(X_test)

# --- 6. FONCTION D'ÉVALUATION ET AFFICHAGE ---

def get_performance_report(Y_true, Y_pred, dataset_name):
    """Calcule et formate les métriques pour le rapport."""
    
    # Séparation des cibles pour les calculs
    y_true_price = Y_true['price'].values
    y_pred_price = Y_pred[:, 0]
    
    y_true_carat = Y_true['carat'].values
    y_pred_carat = Y_pred[:, 1]
    
    results = []
    
    # Métriques pour la Cible 'Price ($)'
    results.append({
        'Dataset': dataset_name,
        'Cible': 'Price ($)',
        'RMSE': np.sqrt(mean_squared_error(y_true_price, y_pred_price)),
        'MAE': mean_absolute_error(y_true_price, y_pred_price),
        'R2 Score': r2_score(y_true_price, y_pred_price)
    })
    
    # Métriques pour la Cible 'Carat (ct)'
    results.append({
        'Dataset': dataset_name,
        'Cible': 'Carat (ct)',
        'RMSE': np.sqrt(mean_squared_error(y_true_carat, y_pred_carat)),
        'MAE': mean_absolute_error(y_true_carat, y_pred_carat),
        'R2 Score': r2_score(y_true_carat, y_pred_carat)
    })
    
    return pd.DataFrame(results)

# Calcul et affichage du rapport
metrics_train = get_performance_report(Y_train, Y_pred_train, "Training")
metrics_test = get_performance_report(Y_test, Y_pred_test, "Test")

final_report = pd.concat([metrics_train, metrics_test], ignore_index=True)
final_report = final_report.sort_values(by='Cible').round(4) # Arrondir pour la comparaison
final_report = final_report.rename(columns={'R2 Score': 'R2 Score'})

# Formatage de l'output exactement comme demandé
print("--- Rapport de Performance (Train vs Test) ---")
print(final_report.to_string(index=False, formatters={'RMSE': '{:.2f}'.format, 'MAE': '{:.2f}'.format, 'R2 Score': '{:.4f}'.format}))

--- Rapport de Performance (Train vs Test) ---
 Dataset      Cible    RMSE     MAE R2 Score
Training Carat (ct)    0.09    0.07   0.9606
    Test Carat (ct)    0.11    0.07   0.9450
Training  Price ($) 1564.09 1200.15   0.8458
    Test  Price ($) 1602.83 1198.42   0.8398
